Description

Embedding Model : text-embedding-3-small

Vector DB : Pinecone

โดยการทดสอบมีวิธีการที่แตกต่างกัน


In [103]:
# import library
import os
import pandas as pd
import time
import uuid
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from openai import AsyncOpenAI
from pinecone import Pinecone, ServerlessSpec
from docx import Document
import fitz
from dotenv import load_dotenv
load_dotenv()

True

In [92]:
# KEY
PINECONE_API_KEY : str
PINECONE_ENV : str
OPENAI_API_KEY : str 
EMBEDDING : str 

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
pc = Pinecone(api_key=PINECONE_API_KEY, environment= PINECONE_ENV)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING = os.getenv("EMBEDDING")
client = AsyncOpenAI(api_key=OPENAI_API_KEY)


In [106]:
# Upload file
start_upload = time.perf_counter()
file_path = "../data"
dfs = []
def read_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

def read_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

for filename in os.listdir(file_path):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(file_path, filename))
        dfs.append(df)
    if filename.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(file_path, filename))
        for sheet_name in df.sheet_names:
            sheet_df = df.parse(sheet_name)
        dfs.append(df)
    if  filename.endswith(".docx"):
        text = read_docx(os.path.join(file_path, filename))
        df = pd.DataFrame({"text": [text]})
        dfs.append(df)
    if filename.endswith(".pdf"):
        text = read_pdf(os.path.join(file_path, filename))
        df = pd.DataFrame({"text": [text]})
        dfs.append(df)
        print(dfs)

if filename.endswith((".csv", ".xlsx")):
  df_combined = pd.concat(dfs, ignore_index=True)
  df_combined.dropna(inplace=True)
  df_combined.drop_duplicates(inplace=True)
  df_combined.reset_index(drop=True, inplace=True)

elif filename.endswith((".pdf", ".docx")):
  df_combined = pd.DataFrame({"text": dfs})

end_upload= time.perf_counter()
upload_time = end_upload - start_upload
print(f"✅ Upload เสร็จ {len(df_combined)} records ในเวลา {upload_time:.2f} วินาที")

print("✅ Shape after cleansing:", df_combined.shape)
# df_combined.head()

✅ Upload เสร็จ 250 records ในเวลา 0.00 วินาที
✅ Shape after cleansing: (250, 11)


Pattern 1

In [78]:
import nest_asyncio
import asyncio
nest_asyncio.apply()

async def embed_batch(batch, embed):
    response = await client.embeddings.create(model=embed, input=batch)
    if hasattr(response, "data") and response.data:
        return [item.embedding for item in response.data]
    else:
        raise ValueError("No 'data' found or 'data' is empty.")

async def batch_process_embedding_async(text_list, embed, batch_size=100):
    tasks = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        tasks.append(embed_batch(batch,embed))
    results = await asyncio.gather(*tasks)
    embeddings = [embedding for batch in results for embedding in batch]
    return embeddings

start_embed_2 = time.perf_counter()
texts = []
metadata_list = []
for i, row in df_combined.iterrows():
    metadata = row.to_dict()
    text = "\n".join([f"{k}: {v}" for k, v in metadata.items()])  # ใช้ key: value
    texts.append(text)
    metadata_list.append((f"vec-{i}", metadata))


embeddings = await batch_process_embedding_async(texts,EMBEDDING)

end_embed_2 = time.perf_counter()
embed_time_2 = end_embed_2 -start_embed_2
print(f"✅ Embedding เสร็จทั้งหมด {len(embeddings)} records ในเวลา {embed_time_2:.2f} วินาที")

✅ Embedding เสร็จทั้งหมด 250 records ในเวลา 3.84 วินาที


In [95]:
import nest_asyncio
import asyncio
nest_asyncio.apply()
async def embed_batch(batch, embed):
    response = await client.embeddings.create(model=embed, input=batch)
    if hasattr(response, "data") and response.data:
        return [item.embedding for item in response.data]
    else:
        raise ValueError("No 'data' found or 'data' is empty.")

async def batch_process_embedding_async(text_list, embed, batch_size=100):
    tasks = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        tasks.append(embed_batch(batch,embed))
    results = await asyncio.gather(*tasks)
    embeddings = [embedding for batch in results for embedding in batch]
    return embeddings

In [96]:
# Prepare vectors for upsert
start_embed_2 = time.perf_counter()
texts = []
metadata_list = []
for i, row in df_combined.iterrows():
    metadata = row.to_dict()
    text = "\n".join([f"{k}: {v}" for k, v in metadata.items()])  # ใช้ key: value
    texts.append(text)
    metadata_list.append((f"vec-{i}", metadata))


embeddings = await batch_process_embedding_async(texts,EMBEDDING)

end_embed_2 = time.perf_counter()
embed_time_2 = end_embed_2 -start_embed_2
print(f"✅ Embedding เสร็จทั้งหมด {len(embeddings)} records ในเวลา {embed_time_2:.2f} วินาที")

✅ Embedding เสร็จทั้งหมด 250 records ในเวลา 2.84 วินาที


In [97]:
async def upsert_batch(index, vectors_batch, namespace):
    try:
        index.upsert(vectors=vectors_batch, namespace=namespace)
    except Exception as e:
        print("❌ Upsert error:", e)

async def parallel_upsert(index, vectors, namespace, batch_size=100, concurrency_limit=5):
    semaphore = asyncio.Semaphore(concurrency_limit)
    async def limited_upsert(batch):
        async with semaphore:
            await upsert_batch(index, batch, namespace)
    tasks = [
        limited_upsert(vectors[i:i + batch_size])
        for i in range(0, len(vectors), batch_size)
    ]
    await asyncio.gather(*tasks)

In [98]:
index_name = "test-1"
namespace = "test-1"

start_upsert_2 = time.perf_counter()
vectors = []
for (vec_id, metadata), embedding, raw_text in zip(metadata_list, embeddings, texts):
    vectors.append({
        "id": vec_id,
        "values": embedding,
        "metadata": {
            **metadata,
            "raw_text": raw_text  # ฝัง raw_text ไว้ใน metadata
        }
    })
index_list = pc.list_indexes().names()
print(f"Existing indexes: {index_list}")

if index_name in index_list:
        print(f"Index '{index_name}' exists. Deleting it...")
        pc.delete_index(index_name)
        print(f"Index '{index_name}' deleted successfully.")


spec = ServerlessSpec(cloud="aws", region=PINECONE_ENV)
pc.create_index(
                name=index_name,
                dimension=1536,  # vector size
                metric="cosine",
                spec=spec
                )
print(f"Index '{index_name}' has been created successfully.")

index = pc.Index(index_name)
await parallel_upsert(index, vectors, namespace, batch_size=100)

end_upsert_2 = time.perf_counter()
upsert_time_2 = end_upsert_2  - start_upsert_2
print(f"✅ Upsert Pinecone เสร็จ {len(vectors)} vectors ในเวลา {upsert_time_2 :.2f} วินาที")

Existing indexes: ['test-1', 'd', 'test-2', 'default-index', 'test12']
Index 'test-1' exists. Deleting it...
Index 'test-1' deleted successfully.
Index 'test-1' has been created successfully.
✅ Upsert Pinecone เสร็จ 250 vectors ในเวลา 22.98 วินาที


In [122]:
from langchain.embeddings import OpenAIEmbeddings
async def retrieve_context_from_pinecone(question: str, index_name: str, namespace: str, top_k: int = 55):
    embedder = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
    question_vector = embedder.embed_query(question)
    index = pc.Index(index_name)
    result = index.query(vector=question_vector, top_k=top_k, include_metadata=True, namespace=namespace)
    context_chunks = []
    for match in result["matches"]:
        raw_text = match["metadata"].get("raw_text", str(match["metadata"]))
        context_chunks.append(raw_text)
    return "\n".join(context_chunks)

In [123]:
import tiktoken

def count_tokens(text, model="text-embedding-3-small"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

question = "สินค้ามีอะไรบ้าง และมีจำนวนเท่าไร"
context = await retrieve_context_from_pinecone(question, index_name, namespace)
num_tokens = count_tokens(context)
print(f"จำนวน token ทั้งหมดใน context: {num_tokens}")

จำนวน token ทั้งหมดใน context: 4851


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
start_q_2 = time.perf_counter()
index_name = "test-2"
namespace = "test-2"

prompt = f"""
ข้อมูลต่อไปนี้มาจากไฟล์ในรูปแบบตาราง เช่น CSV หรือ Excel ซึ่งอาจมีหลายประเภทข้อมูลและหลายแถว:
{context}

คำถามของฉันคือ: "{question}"

กรุณาตอบโดย:
- วิเคราะห์ข้อมูลทั้งหมดให้ครบถ้วน
- สรุปคำตอบโดยใช้ภาษาธรรมดา ชัดเจน และไม่ใช้การจัดรูปแบบตัวหนา หัวข้อ หรือสัญลักษณ์พิเศษ เช่น ** หรือ -
- หากมีหลายรายการสินค้า ให้ระบุชื่อสินค้า พร้อมจำนวน และหน่วยตามลักษณะของสินค้า เช่น เครื่อง, ตัว, คู่, เล่ม ฯลฯ
- หากต้องรวมจำนวน ให้รวมและแสดงยอดรวมทั้งหมด
- ตอบเป็นภาษาไทยแบบเป็นธรรมชาติ เข้าใจง่าย เหมือนอธิบายให้คนทั่วไปฟัง
- หากไม่พบข้อมูลที่เกี่ยวข้อง ให้ตอบว่า "ไม่พบข้อมูลที่เกี่ยวข้อง"
"""

llm = ChatOpenAI(temperature=0, model="gpt-4",api_key=OPENAI_API_KEY)
response = llm.invoke(prompt)


end_q_2 = time.perf_counter()
response_time_2 = end_q_2 - start_q_2

print("คำตอบ:", response.content)
print(f"⏱ ใช้เวลาในการตอบ: {response_time_2:.2f} วินาที")

คำตอบ: จากการวิเคราะห์ข้อมูล พบว่ามีสินค้าทั้งหมด 6 ประเภท ดังนี้:

1. ยีนส์ (Jeans) จำนวน 8 ตัว
2. ตู้เย็น (Refrigerator) จำนวน 9 เครื่อง
3. สมาร์ทโฟน (Smartphone) จำนวน 14 เครื่อง
4. นาฬิกาอัจฉริยะ (Smartwatch) จำนวน 16 เรือน
5. เครื่องซักผ้า (Washing Machine) จำนวน 14 เครื่อง
6. เสื้อยืด (T-Shirt) จำนวน 16 ตัว

นอกจากนี้ยังมีสินค้าประเภทอื่นๆ อีก 3 ประเภท ได้แก่ รองเท้าวิ่ง (Running Shoes) จำนวน 7 คู่, หูฟัง (Headphones) จำนวน 2 คู่ และ แล็ปท็อป (Laptop) จำนวน 16 เครื่อง

ดังนั้น สรุปได้ว่ามีสินค้าทั้งหมด 9 ประเภท และจำนวนรวมทั้งหมดคือ 92 รายการ
⏱ ใช้เวลาในการตอบ: 18.32 วินาที


In [125]:
print(f"✅ Embedding เสร็จทั้งหมด {len(embeddings)} records ในเวลา {embed_time_2:.2f} วินาที")
print(f"✅ Upsert Pinecone เสร็จ {len(vectors)} vectors ในเวลา {upsert_time_2:.2f} วินาที")
print(f"⏱ ใช้เวลาในการตอบ: {response_time_2:.2f} วินาที")
print(f"เวลาโดยรวมupload pineconeทั้งหมด : {embed_time_2+upsert_time_2:.2f} ")

✅ Embedding เสร็จทั้งหมด 250 records ในเวลา 2.84 วินาที
✅ Upsert Pinecone เสร็จ 250 vectors ในเวลา 22.98 วินาที
⏱ ใช้เวลาในการตอบ: 18.32 วินาที
เวลาโดยรวมupload pineconeทั้งหมด : 25.82 


Pattern 2 

In [107]:
import nest_asyncio
import asyncio
nest_asyncio.apply()
async def embed_batch(batch, embed):
    response = await client.embeddings.create(model=embed, input=batch)
    return [item.embedding for item in response.data]

async def embed_all_rows(df,embed ,batch_size=100):
    texts = [" ".join(map(str, row.values)) for _, row in df.iterrows()]
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        embeddings = await embed_batch(batch,embed)
        all_embeddings.extend(embeddings)

    return all_embeddings

start_embed_time = time.perf_counter()
embeddings = asyncio.run(embed_all_rows(df_combined,EMBEDDING))
end_embed_time = time.perf_counter()
embed_time = end_embed_time  - start_embed_time

print(f"✅ Embedding เสร็จทั้งหมด {len(embeddings)} records ในเวลา {embed_time:.2f} วินาที")

✅ Embedding เสร็จทั้งหมด 250 records ในเวลา 6.85 วินาที


In [109]:
start_upsert = time.perf_counter()
# ลบ index เดิมถ้ามี
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

spec = ServerlessSpec(cloud="aws", region=PINECONE_ENV)
# pc.create_index(name=index_name, dimension=1536, metric="cosine", spec=spec)
pc.create_index(index_name, dimension=len(embeddings[0]), metric="cosine", spec=spec)
index = pc.Index(index_name)

# สร้าง vectors
vectors = [{
    "id": f"vec-{i}",
    "values": embeddings[i],
    "metadata": df_combined.iloc[i].to_dict()
} for i in range(len(embeddings))]

# ⏱ จับเวลา upsert
index.upsert(vectors=vectors, namespace=namespace)
end_upsert = time.perf_counter()
upsert_time =  end_upsert - start_upsert

print(f"✅ Upsert Pinecone เสร็จ {len(vectors)} vectors ในเวลา {upsert_time :.2f} วินาที")


✅ Upsert Pinecone เสร็จ 250 vectors ในเวลา 24.52 วินาที


In [110]:
async def retrieve_context_from_pinecone(question: str, index_name: str, namespace: str, top_k: int = 50):
    embedder = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    question_vector = embedder.embed_query(question)

    index = pc.Index(index_name)
    result = index.query(
        vector=question_vector,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace
    )

    context_chunks = []
    for match in result['matches']:
        metadata = match['metadata']
        context_chunks.append(str(metadata))

    return "\n".join(context_chunks)

In [116]:
import tiktoken

def count_tokens(text, model="text-embedding-3-small"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))
question = "สินค้ามีอะไรบ้าง และมีจำนวนเท่าไร"
context = await retrieve_context_from_pinecone(question, index_name, namespace)
num_tokens_p2 = count_tokens(context)
print(f"จำนวน token ทั้งหมดใน context: {num_tokens_p2}")

จำนวน token ทั้งหมดใน context: 4401


In [112]:
from langchain_openai import ChatOpenAI
start_q = time.perf_counter()

question = "สินค้ามีอะไรบ้าง และมีจำนวนเท่าไร"
context = await retrieve_context_from_pinecone(question, index_name, namespace)
prompt = f"""
ข้อมูลต่อไปนี้มาจากไฟล์ในรูปแบบตาราง เช่น CSV หรือ Excel ซึ่งอาจมีหลายประเภทข้อมูลและหลายแถว:
{context}

คำถามของฉันคือ: "{question}"

กรุณาตอบโดย:
- วิเคราะห์ข้อมูลทั้งหมดให้ครบถ้วน
- สรุปคำตอบโดยใช้ภาษาธรรมดา ชัดเจน และไม่ใช้การจัดรูปแบบตัวหนา หัวข้อ หรือสัญลักษณ์พิเศษ เช่น ** หรือ -
- หากมีหลายรายการสินค้า ให้ระบุชื่อสินค้า พร้อมจำนวน และหน่วยตามลักษณะของสินค้า เช่น เครื่อง, ตัว, คู่, เล่ม ฯลฯ
- หากต้องรวมจำนวน ให้รวมและแสดงยอดรวมทั้งหมด
- ตอบเป็นภาษาไทยแบบเป็นธรรมชาติ เข้าใจง่าย เหมือนอธิบายให้คนทั่วไปฟัง
- หากไม่พบข้อมูลที่เกี่ยวข้อง ให้ตอบว่า "ไม่พบข้อมูลที่เกี่ยวข้อง"
"""

llm = ChatOpenAI(temperature=0, model="gpt-4-turbo",api_key=OPENAI_API_KEY)
response = llm.invoke(prompt)


end_q = time.perf_counter()
response_time = end_q - start_q

print("คำตอบ:", response.content)
print(f"⏱ ใช้เวลาในการตอบ: {response_time:.2f} วินาที")

คำตอบ: จากข้อมูลที่ได้รับ พบว่ามีสินค้าหลายประเภทที่ถูกจำหน่าย โดยมีรายละเอียดดังนี้:

1. แล็ปท็อป (Laptop) จำนวน 26 เครื่อง
2. รองเท้าวิ่ง (Running Shoes) จำนวน 24 คู่
3. หนังสือ (Book) จำนวน 20 เล่ม
4. หูฟัง (Headphones) จำนวน 23 คู่
5. เสื้อยืด (T-Shirt) จำนวน 12 ตัว
6. สมาร์ทวอทช์ (Smartwatch) จำนวน 20 เรือน
7. สมาร์ทโฟน (Smartphone) จำนวน 18 เครื่อง
8. ตู้เย็น (Refrigerator) จำนวน 4 เครื่อง

รวมทั้งหมดมีสินค้า 147 รายการ จากหลายหมวดหมู่ที่ถูกจำหน่ายในข้อมูลที่ได้รับ.
⏱ ใช้เวลาในการตอบ: 14.70 วินาที


In [114]:
print(f"✅ Embedding เสร็จทั้งหมด {len(embeddings)} records ในเวลา {embed_time:.2f} วินาที")
print(f"✅ Upsert Pinecone เสร็จ {len(vectors)} vectors ในเวลา {upsert_time :.2f} วินาที")
print(f"⏱ ใช้เวลาในการตอบ: {response_time:.2f} วินาที")
print(f"เวลาโดยรวมupload pineconeทั้งหมด : {embed_time+upsert_time:.2f} ")

✅ Embedding เสร็จทั้งหมด 250 records ในเวลา 6.85 วินาที
✅ Upsert Pinecone เสร็จ 250 vectors ในเวลา 24.52 วินาที
⏱ ใช้เวลาในการตอบ: 14.70 วินาที
เวลาโดยรวมupload pineconeทั้งหมด : 31.37 
